# **Setup, install dependencies**

In [ ]:
# Clone repo and install requirements

!git clone https://github.com/suragnair/alpha-zero-general.git

Cloning into 'alpha-zero-general'...
remote: Enumerating objects: 1045, done.
remote: Total 1045 (delta 0), reused 0 (delta 0), pack-reused 1045
Receiving objects: 100% (1045/1045), 423.63 MiB | 24.50 MiB/s, done.
Resolving deltas: 100% (575/575), done.
Checking out files: 100% (109/109), done.


In [ ]:
%cd '/content/alpha-zero-general'

/content/alpha-zero-general


In [ ]:
!git checkout -t origin/master

Branch 'dotsandboxes' set up to track remote branch 'dotsandboxes' from 'origin'.
Switched to a new branch 'dotsandboxes'


In [ ]:
!pip install -r docker/requirements.txt

     |████████████████████████████████| 430kB 8.7MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 3.4MB 19.0MB/s 
     |████████████████████████████████| 102kB 13.2MB/s 
     |████████████████████████████████| 450kB 64.9MB/s 
     |████████████████████████████████| 15.0MB 113kB/s 
     |████████████████████████████████| 12.2MB 14.0MB/s 
     |████████████████████████████████| 11.8MB 60.0MB/s 
     |████████████████████████████████| 31.2MB 80kB/s 
     |████████████████████████████████| 12.4MB 49.9MB/s 
     |████████████████████████████████| 25.3MB 130kB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 92kB 11.7MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=87717a63bc2a4a0d7ba29a

# **Train AlphaZero**

In [ ]:
import logging
import coloredlogs
from Coach import Coach
from utils import dotdict
from dotsandboxes.keras.NNet import NNetWrapper
from dotsandboxes.DotsAndBoxesGame import DotsAndBoxesGame

In [ ]:
log = logging.getLogger(__name__)
coloredlogs.install(level='INFO')  # Change this to DEBUG to see more info.

In [ ]:
args = dotdict({
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 25,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,
    'checkpoint': './temp/',
    'load_model': False,
    'numItersForTrainExamplesHistory': 20,
})

In [ ]:
# If you have a pre-trained model, you can load it here.
import os
if os.path.exists(os.path.join('pretrained_models', 'dotsandboxes', 'keras', '3x3', 'best.pth.tar.index')):
  print ("Using best pre-existing model")
  args['load_model'] = True
  args['load_folder_file'] = ('pretrained_models/dotsandboxes/keras/3x3','best.pth.tar')
else:
  print ("Not using best pre-existing model")

Using best pre-existing model


In [ ]:
# Set very low iterations to let this notebook run in its entirety.

# In reality, training a model, even as simple as the one for Dots and Boxes, can take several hours or days.
args['numIters'] = 2
args['numEps'] = 2
args['arenaCompare'] = 2

In [ ]:
game = DotsAndBoxesGame(n=3)

In [ ]:
nnet = NNetWrapper(game)

In [ ]:
if args.load_model:
    print('Loading checkpoint "{}/{}"...'.format(args.load_folder_file[0], args.load_folder_file[1]))
    nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
else:
    print('Not loading a checkpoint.')

Loading checkpoint "pretrained_models/dotsandboxes/keras/3x3/best.pth.tar"...


In [ ]:
coach = Coach(game, nnet, args)

In [ ]:
%time coach.learn()

2020-11-08 02:05:04 dece90a6a033 Coach[66] INFO Starting Iter #1 ...
Self Play: 100%|██████████| 2/2 [00:25<00:00, 12.62s/it]


Checkpoint Directory exists! 
Epoch 1/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5881 - pi_loss: 0.5198 - v_loss: 0.0683
Epoch 2/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5992 - pi_loss: 0.5420 - v_loss: 0.0572
Epoch 3/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5989 - pi_loss: 0.5388 - v_loss: 0.0601
Epoch 4/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5511 - pi_loss: 0.5118 - v_loss: 0.0393
Epoch 5/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5730 - pi_loss: 0.5315 - v_loss: 0.0415
Epoch 6/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5990 - pi_loss: 0.5565 - v_loss: 0.0426
Epoch 7/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5594 - pi_loss: 0.5094 - v_loss: 0.0500
Epoch 8/10
8/8 [==============================] - 0s 4ms/step - loss: 0.5947 - pi_loss: 0.5444 - v_loss: 0.0503
Epoch 9/10
8/8 [==============================] - 0s 4ms/step - loss: 0.54

2020-11-08 02:05:31 dece90a6a033 Coach[66] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (2): 100%|██████████| 1/1 [00:12<00:00, 13.00s/it]
2020-11-08 02:05:59 dece90a6a033 Coach[66] INFO NEW/PREV WINS : 1 / 1 ; DRAWS : 0
2020-11-08 02:05:59 dece90a6a033 Coach[66] INFO REJECTING NEW MODEL
2020-11-08 02:05:59 dece90a6a033 Coach[66] INFO Starting Iter #2 ...
Self Play: 100%|██████████| 2/2 [00:21<00:00, 10.60s/it]


Checkpoint Directory exists! 
Epoch 1/10
16/16 [==============================] - 0s 4ms/step - loss: 0.5865 - pi_loss: 0.5488 - v_loss: 0.0377
Epoch 2/10
16/16 [==============================] - 0s 4ms/step - loss: 0.5419 - pi_loss: 0.5215 - v_loss: 0.0205
Epoch 3/10
16/16 [==============================] - 0s 3ms/step - loss: 0.5158 - pi_loss: 0.5012 - v_loss: 0.0145
Epoch 4/10
16/16 [==============================] - 0s 3ms/step - loss: 0.5684 - pi_loss: 0.5355 - v_loss: 0.0329
Epoch 5/10
16/16 [==============================] - 0s 3ms/step - loss: 0.5401 - pi_loss: 0.5246 - v_loss: 0.0155
Epoch 6/10
16/16 [==============================] - 0s 3ms/step - loss: 0.5431 - pi_loss: 0.5248 - v_loss: 0.0183
Epoch 7/10
16/16 [==============================] - 0s 4ms/step - loss: 0.5400 - pi_loss: 0.5213 - v_loss: 0.0187
Epoch 8/10
16/16 [==============================] - 0s 4ms/step - loss: 0.5285 - pi_loss: 0.5120 - v_loss: 0.0165
Epoch 9/10
16/16 [==============================] - 0s 4ms

2020-11-08 02:06:21 dece90a6a033 Coach[66] INFO PITTING AGAINST PREVIOUS VERSION
Arena.playGames (2): 100%|██████████| 1/1 [00:11<00:00, 11.99s/it]
2020-11-08 02:06:47 dece90a6a033 Coach[66] INFO NEW/PREV WINS : 1 / 1 ; DRAWS : 0
2020-11-08 02:06:47 dece90a6a033 Coach[66] INFO REJECTING NEW MODEL


CPU times: user 1min 39s, sys: 6.36 s, total: 1min 46s
Wall time: 1min 43s


In [ ]:
# Checkpoints and best model (if found) will be saved in this folder
%ls /content/alpha-zero-general/temp

checkpoint                     temp.pth.tar.data-00000-of-00001
checkpoint_0.pth.tar.examples  temp.pth.tar.index
checkpoint_1.pth.tar.examples
